# 10 - Pareamento

## O Que a Regressão Está Fazendo, Afinal?

Até agora, vimos que a regressão realiza um trabalho incrível ao controlar variáveis adicionais quando fazemos uma comparação entre teste e controle. Se tivermos independência, $(Y_0, Y_1)\perp T | X$, então a regressão pode identificar o ATE ao controlar X. A forma como a regressão realiza isso é meio mágica. Para ter uma intuição sobre isso, vamos lembrar do caso em que todas as variáveis X são variáveis dummy. Se esse for o caso, a regressão divide os dados em células dummy e calcula a diferença média entre teste e controle. Essa diferença de médias mantém os valores de X constantes, já que estamos fazendo isso em uma célula fixa da variável dummy X. É como se estivéssemos fazendo $E[Y|T=1] - E[Y|T=0] | X=x$, onde $x$ é uma célula dummy (todas as dummies definidas como 1, por exemplo). A regressão então combina a estimativa em cada uma das células para produzir um ATE final. A forma como ela faz isso é aplicando pesos às células proporcionais à variância do tratamento nesse grupo.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf

import graphviz as gr

%matplotlib inline

style.use("fivethirtyeight")

Para dar um exemplo, suponhamos que estou tentando estimar o efeito de um medicamento e tenho 6 homens e 4 mulheres. Minha variável de resposta é o número de dias hospitalizados, e espero que o medicamento possa reduzir esse período. Nos homens, o verdadeiro efeito causal é -3, ou seja, o medicamento reduz o período de internação em 3 dias. Nas mulheres, é -2. Para tornar as coisas mais interessantes, os homens são muito mais afetados por essa doença e ficam mais tempo no hospital. Além disso, recebem muito mais do medicamento. Apenas 1 dos 6 homens não recebe o medicamento. Por outro lado, as mulheres são mais resistentes a essa doença, então ficam menos tempo no hospital. 50% das mulheres recebem o medicamento.

In [2]:
drug_example = pd.DataFrame(dict(
    sex= ["M","M","M","M","M","M", "W","W","W","W"],
    drug=[1,1,1,1,1,0,  1,0,1,0],
    days=[5,5,5,5,5,8,  2,4,2,4]
))

Observe que a simples comparação entre tratamento e controle resulta em um efeito negativamente enviesado, ou seja, o medicamento parece ser menos eficaz do que realmente é. Isso é esperado, uma vez que omitimos a variável confundidora sexo. Neste caso, o ATE estimado é menor do que o verdadeiro, porque os homens recebem mais do medicamento e são mais afetados pela doença.

In [3]:
drug_example.query("drug==1")["days"].mean() - drug_example.query("drug==0")["days"].mean()

-1.1904761904761898

Como o efeito verdadeiro para os homens é -3 e o efeito verdadeiro para as mulheres é -2, o ATE deveria ser

$
ATE=\dfrac{(-3*6) + (-2*4)}{10}=-2.6
$

Esta estimativa é feita por 1) dividir os dados em células de confundidores, neste caso, homens e mulheres, 2) estimar o efeito em cada célula e 3) combinar a estimativa com uma média ponderada, onde o peso é o tamanho da amostra da célula ou grupo de covariáveis. Se tivéssemos exatamente o mesmo número de homens e mulheres nos dados, a estimativa do ATE estaria bem no meio do ATE dos 2 grupos, -2.5. Como há mais homens do que mulheres em nosso conjunto de dados, a estimativa do ATE está um pouco mais próxima do ATE dos homens. Isso é chamado de estimativa não paramétrica, uma vez que não faz nenhuma suposição sobre como os dados foram gerados.

Se controlarmos o sexo usando regressão, adicionaremos a suposição de linearidade. A regressão também dividirá os dados em homens e mulheres e estimará o efeito em ambos esses grupos. Até aqui, tudo bem. No entanto, quando se trata de combinar o efeito em cada grupo, ela não os pondera pelo tamanho da amostra. Em vez disso, a regressão utiliza pesos proporcionais à variância do tratamento nesse grupo. Em nosso caso, a variância do tratamento em homens é menor do que em mulheres, já que apenas um homem está no grupo de controle. Para ser exato, a variância de T para homens é $0.139=1/6*(1 - 1/6)$ e para mulheres é $0.25=2/4*(1 - 2/4)$. Portanto, a regressão dará um peso maior para mulheres em nosso exemplo, e o ATE estará um pouco mais próximo do ATE das mulheres, que é -2.

In [4]:
smf.ols('days ~ drug + C(sex)', data=drug_example).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,7.5455,0.188,40.093,0.000,7.100,7.990
C(sex)[T.W],-3.3182,0.176,-18.849,0.000,-3.734,-2.902
drug,-2.4545,0.188,-13.042,0.000,-2.900,-2.010


Esse resultado é mais intuitivo com variáveis dummy, mas, de uma maneira estranha, a regressão também mantém variáveis contínuas constantes enquanto estima o efeito. Também com variáveis contínuas, o ATE apontará na direção onde as covariáveis têm mais variância.

Portanto, vimos que a regressão tem suas idiossincrasias. Ela é linear, paramétrica, gosta de características com alta variância... Isso pode ser bom ou ruim, dependendo do contexto. Por causa disso, é importante estar ciente de outras técnicas que podemos usar para controlar confundidores. Não apenas são uma ferramenta adicional em seu arsenal causal, mas entender diferentes maneiras de lidar com confundidores expande nossa compreensão do problema. Por esse motivo, apresento a você agora o **Estimador de Subclassificação!**


## O Estimador de Subclassificação

![img](./data/img/matching/explain.png)

Se houver algum efeito causal que desejamos estimar, como o efeito do treinamento profissional nos ganhos, **e** o tratamento não for atribuído aleatoriamente, precisamos ter cuidado com os confundidores. Pode ser que apenas pessoas mais motivadas façam o treinamento e que teriam ganhos mais altos independentemente do treinamento. Precisamos estimar o efeito do programa de treinamento dentro de pequenos grupos de indivíduos que sejam aproximadamente iguais em nível de motivação e quaisquer outros confundidores que possamos ter.

Mais geralmente, se houver algum efeito causal que desejamos estimar, mas é difícil fazê-lo devido ao confundimento de algumas variáveis X, o que precisamos fazer é comparar tratamento versus controle dentro de pequenos grupos onde X é o mesmo. Se tivermos independência condicional $(Y_0, Y_1)\perp T | X$, então podemos escrever o ATE da seguinte forma.

$
ATE = \int(E[Y|X, T=1] - E[Y|X, T=0])dP(x)
$

O que esta integral faz é percorrer todo o espaço da distribuição das características X, calcular a diferença nas médias para todos esses pequenos espaços e combinar tudo no ATE. Outra maneira de ver isso é pensar em um conjunto discreto de características. Nesse caso, podemos dizer que as características X assumem K células diferentes $\{X_1, X_2, ..., X_k\}$ e o que estamos fazendo é calcular o efeito do tratamento em cada célula e combiná-los no ATE. Neste caso discreto, convertendo a integral em uma soma, podemos derivar o estimador de subclassificações.


$
\hat{ATE} = \sum^K_{k=1}(\bar{Y}_{k1} - \bar{Y}_{k0}) * \dfrac{N_k}{N}
$

onde a barra representa a média do resultado nos tratados, $Y_{k1}$, e não tratados, $Y_{k0}$, na célula k, e $N_{k}$ é o número de observações na mesma célula. Como você pode ver, estamos calculando um ATE local para cada célula e os combinando usando uma média ponderada, onde os pesos são o tamanho da amostra da célula. No nosso exemplo da medicina acima, isso seria a primeira estimativa, que nos deu -2.6.

## Estimador de Pareamento

![img](./data/img/matching/its-a-match.png)

O estimador de subclassificações não é muito utilizado na prática (veremos o motivo em breve, é por causa da maldição da dimensionalidade), mas ele nos proporciona uma boa intuição sobre o que um estimador de inferência causal deveria fazer, como deveria controlar confundidores. Isso nos permite explorar outros tipos de estimadores, como o Estimador de Pareamento.

A ideia é muito semelhante. Como alguma variável de confusão X faz com que os tratados e não tratados não sejam inicialmente comparáveis, posso torná-los comparáveis **associando cada unidade tratada a uma unidade não tratada semelhante**. É como se eu estivesse encontrando um "gêmeo" não tratado para cada unidade tratada. Ao fazer essas comparações, tratados e não tratados tornam-se comparáveis novamente.

Como exemplo, suponhamos que estamos tentando estimar o efeito de um programa de treinee nos salários. Aqui está como os participantes do treinee se parecem:

In [5]:
trainee = pd.read_csv("./data/trainees.csv")
trainee.query("trainees==1")

,unit,trainees,age,earnings
0,1,1,28,17700
1,2,1,34,10200
2,3,1,29,14400
3,4,1,25,20800
4,5,1,29,6100
5,6,1,23,28600
6,7,1,33,21900
7,8,1,27,28800
8,9,1,31,20300
9,10,1,26,28100


E aqui estão os não-trainees:

In [6]:
trainee.query("trainees==0")

,unit,trainees,age,earnings
19,20,0,43,20900
20,21,0,50,31000
21,22,0,30,21000
22,23,0,27,9300
23,24,0,54,41100
24,25,0,48,29800
25,26,0,39,42000
26,27,0,28,8800
27,28,0,24,25500
28,29,0,33,15500


Se eu fizer uma simples comparação de médias, veremos que os participantes do programa de trainee ganham menos dinheiro do que aqueles que não passaram pelo programa.

In [7]:
trainee.query("trainees==1")["earnings"].mean() - trainee.query("trainees==0")["earnings"].mean()

-4297.49373433584

No entanto, se olharmos para a tabela acima, percebemos que os participantes do programa de trainee são muito mais jovens do que os não participantes, o que indica que a idade é provavelmente um confundidor. Vamos usar o pareamento pela idade para tentar corrigir isso. Vamos pegar a unidade 1 tratada e pareá-la com a unidade 27, já que ambas têm 28 anos. A unidade 2 será pareada com a unidade 34, a unidade 3 com a unidade 37, a unidade 4 será pareada com a unidade 35... Quando se trata da unidade 5, precisamos encontrar alguém com 29 anos nos não tratados, mas essa é a unidade 37, que já está pareada. Isso não é um problema, pois podemos usar a mesma unidade várias vezes. Se mais de uma unidade for uma correspondência, podemos escolher aleatoriamente entre elas.

É assim que o conjunto de dados pareado se parece para as primeiras 7 unidades.

In [8]:
# make dataset where no one has the same age
unique_on_age = (trainee
                 .query("trainees==0")
                 .drop_duplicates("age"))

matches = (trainee
           .query("trainees==1")
           .merge(unique_on_age, on="age", how="left", suffixes=("_t_1", "_t_0"))
           .assign(t1_minuts_t0 = lambda d: d["earnings_t_1"] - d["earnings_t_0"]))

matches.head(7)

,unit_t_1,trainees_t_1,age,earnings_t_1,unit_t_0,trainees_t_0,earnings_t_0,t1_minuts_t0
0,1,1,28,17700,27,0,8800,8900
1,2,1,34,10200,34,0,24200,-14000
2,3,1,29,14400,37,0,6200,8200
3,4,1,25,20800,35,0,23300,-2500
4,5,1,29,6100,37,0,6200,-100
5,6,1,23,28600,40,0,9500,19100
6,7,1,33,21900,29,0,15500,6400


Observe como a última coluna tem a diferença nos ganhos entre o tratado e sua unidade não tratada pareada. Se tirarmos a média desta última coluna, obtemos a estimativa do ATET controlando para a idade. Observe como a estimativa agora é muito positiva, em comparação com a anterior, onde usamos uma simples diferença nas médias.

In [9]:
matches["t1_minuts_t0"].mean()

2457.8947368421054

Mas esse foi um exemplo muito forçado, apenas para introduzir o pareamento. Na realidade, geralmente temos mais de uma característica e as unidades não se parecem perfeitamente. Nesse caso, precisamos definir alguma medida de proximidade para comparar o quão próximas as unidades estão umas das outras. Uma métrica comum para isso é a norma euclidiana $||X_i - X_j||$. No entanto, essa diferença não é invariante em relação à escala das características. Isso significa que características como idade, que assumem valores nas casas decimais, serão muito menos importantes ao calcular essa norma em comparação com características como renda, que assumem valores na ordem das centenas. Por esse motivo, antes de aplicar a norma, precisamos escalonar as características para que estejam em uma escala aproximadamente semelhante.

Tendo definido uma medida de distância, agora podemos definir a correspondência como o vizinho mais próximo dessa amostra que desejamos parear. Em termos matemáticos, podemos escrever o estimador de pareamento da seguinte maneira:

$
\hat{ATE} = \frac{1}{N} \sum^N_{i=1} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)
$

Onde $Y_{jm}(i)$ é a amostra do outro grupo de tratamento que é mais semelhante a $Y_i$. Fazemos isso $2T_i - 1$ vezes para parear nos dois sentidos: tratados com controles e controles com tratados.

Para testar este estimador, vamos considerar um exemplo de medicamento. Mais uma vez, queremos encontrar o efeito de um medicamento nos dias até a recuperação. Infelizmente, esse efeito é confundido por gravidade, sexo e idade. Temos razões para acreditar que os pacientes com condições mais graves têm uma chance maior de receber o medicamento.

In [10]:
med = pd.read_csv("./data/medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


Se olharmos para uma simples diferença nas médias, $E[Y|T=1]-E[Y|T=0]$, veremos que o tratamento leva, em média, 16,9 dias a mais para a recuperação em comparação com o não tratado. Isso provavelmente é devido a confundimento, já que não esperamos que o medicamento cause danos ao paciente.

In [11]:
med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean()

16.895799546498726

Para corrigir esse viés, vamos controlar para X usando o pareamento. Primeiro, precisamos lembrar de escalonar nossas características, caso contrário, características como idade terão uma importância maior do que características como gravidade ao calcular a distância entre os pontos. Para fazer isso, podemos padronizar as características.

In [12]:
# scale features
X = ["severity", "age", "sex"]
y = "recovery"

med = med.assign(**{f: (med[f] - med[f].mean())/med[f].std() for f in X})
med.head()

,sex,age,severity,medication,recovery
0,-0.996980,0.280787,1.459800,1,31
1,1.002979,0.865375,1.502164,1,49
2,1.002979,-0.338749,0.057796,0,38
3,1.002979,0.399465,-0.512557,0,35
4,-0.996980,-0.610473,-0.911125,0,15


Agora, quanto ao pareamento em si. Em vez de codificar uma função de pareamento, usaremos o algoritmo de K-vizinhos mais próximos *(KNN, do inglês "k-Nearest Neighbors")* do [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html). Este algoritmo faz previsões encontrando o ponto de dados mais próximo em um conjunto de estimativa ou treinamento.

Para o pareamento, precisaremos de dois desses. Um, `mt0`, armazenará os pontos não tratados e encontrará correspondências nos não tratados quando solicitado. O outro, `mt1`, armazenará o ponto tratado e encontrará correspondências nos tratados quando solicitado. Após esta etapa de ajuste, podemos usar esses modelos KNN para fazer previsões, que serão nossas correspondências.

In [13]:
from sklearn.neighbors import KNeighborsRegressor

treated = med.query("medication==1")
untreated = med.query("medication==0")

mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[X], untreated[y])
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[X], treated[y])

predicted = pd.concat([
    # find matches for the treated looking at the untreated knn model
    treated.assign(match=mt0.predict(treated[X])),
    
    # find matches for the untreated looking at the treated knn model
    untreated.assign(match=mt1.predict(untreated[X]))
])

predicted.head()

,sex,age,severity,medication,recovery,match
0,-0.996980,0.280787,1.459800,1,31,39.0
1,1.002979,0.865375,1.502164,1,49,52.0
7,-0.996980,1.495134,1.268540,1,38,46.0
10,1.002979,-0.106534,0.545911,1,34,45.0
16,-0.996980,0.043034,1.428732,1,30,39.0


Com as correspondências, agora podemos aplicar a fórmula do estimador de pareamento.

$
\hat{ATE} = \frac{1}{N} \sum^N_{i=1} (2T_i - 1)\big(Y_i - Y_{jm}(i)\big)
$


In [14]:
np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"]))

-0.9954

Usando esse tipo de pareamento, podemos ver que o efeito do medicamento não é mais positivo. Isso significa que, controlando para X, o medicamento reduz o tempo de recuperação em cerca de 1 dia, em média. Isso já é uma melhoria significativa em relação à estimativa enviesada que previa um aumento de 16,9 dias no tempo de recuperação.

No entanto, ainda podemos melhorar.

## Viés de Pareamento

Acontece que o estimador de pareamento como o projetamos acima é enviesado. Para ver isso, consideremos o estimador ATET, em vez do ATE, apenas porque é mais simples de escrever. A intuição se aplicará ao ATE também.

$
\hat{ATET} = \frac{1}{N_1}\sum (Y_i - Y_j(i))
$

onde $N_1$ é o número de indivíduos tratados e $Y_j(i)$ é a correspondência não tratada da unidade tratada i. Para verificar o viés, o que fazemos é esperar poder aplicar o Teorema do Limite Central para que convirja para uma distribuição normal com média zero.

$
\sqrt{N_1}(\hat{ATET} - ATET)
$

No entanto, isso nem sempre acontece. Se definirmos a média do resultado para os não tratados dados X, $\mu_0(x)=E[Y|X=x, T=0]$, teremos que (aliás, omiti a prova disso porque é um pouco além do ponto aqui).

$
E[\sqrt{N_1}(\hat{ATET} - ATET)] = E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]
$

Agora, $\mu_0(X_i) - \mu_0(X_j(i))$ não é tão simples de entender, então vamos analisá-lo mais cuidadosamente. $\mu_0(X_i)$ é o valor do resultado Y de uma unidade tratada $i$ se ela não tivesse sido tratada. Portanto, é o resultado contrafactual $Y_0$ para a unidade i. $\mu_0(X_j(i))$ é o resultado da unidade não tratada $j$ que é a correspondência da unidade $i$. Portanto, é também o $Y_0$, mas agora para a unidade $j$. Só que desta vez, é um resultado factual, porque $j$ está no grupo não tratado. Agora, como $j$ e $i$ são apenas semelhantes, mas não iguais, isso provavelmente não será zero. Em outras palavras, $X_i \approx X_j$. Portanto, $Y_{0i} \approx Y_{0j}$.

À medida que aumentamos o tamanho da amostra, haverá mais unidades para parear, então a diferença entre a unidade $i$ e sua correspondência $j$ também ficará menor. Mas essa diferença converge para zero lentamente. Como resultado, $E[\sqrt{N_1}(\mu_0(X_i) - \mu_0(X_j(i)))]$ pode não convergir para zero, porque $\sqrt{N_1}$ cresce mais rápido do que $(\mu_0(X_i) - \mu_0(X_j(i)))$ diminui.

O viés surge quando as discrepâncias de pareamento são grandes. Felizmente, sabemos como corrigi-lo. Cada observação contribui com $(\mu_0(X_i) - \mu_0(X_j(i)))$ para o viés, então tudo o que precisamos fazer é subtrair essa quantidade de cada comparação de pareamento em nosso estimador. Para fazer isso, podemos substituir $\mu_0(X_j(i))$ por algum tipo de estimativa dessa quantidade $\hat{\mu_0}(X_j(i))$, que pode ser obtida com modelos como regressão linear. Isso atualiza o estimador ATET para a seguinte equação:

$
\hat{ATET} = \frac{1}{N_1}\sum \big((Y_i - Y_{j(i)}) - (\hat{\mu_0}(X_i) - \hat{\mu_0}(X_{j(i)}))\big)
$

onde $\hat{\mu_0}(x)$ é alguma estimativa de $E[Y|X, T=0]$, como uma regressão linear ajustada apenas na amostra não tratada.

In [15]:
from sklearn.linear_model import LinearRegression

# fit the linear regression model to estimate mu_0(x)
ols0 = LinearRegression().fit(untreated[X], untreated[y])
ols1 = LinearRegression().fit(treated[X], treated[y])

# find the units that match to the treated
treated_match_index = mt0.kneighbors(treated[X], n_neighbors=1)[1].ravel()

# find the units that match to the untreatd
untreated_match_index = mt1.kneighbors(untreated[X], n_neighbors=1)[1].ravel()

predicted = pd.concat([
    (treated
     # find the Y match on the other group
     .assign(match=mt0.predict(treated[X])) 
     
     # build the bias correction term
     .assign(bias_correct=ols0.predict(treated[X]) - ols0.predict(untreated.iloc[treated_match_index][X]))),
    (untreated
     .assign(match=mt1.predict(untreated[X]))
     .assign(bias_correct=ols1.predict(untreated[X]) - ols1.predict(treated.iloc[untreated_match_index][X])))
])

predicted.head()

,sex,age,severity,medication,recovery,match,bias_correct
0,-0.996980,0.280787,1.459800,1,31,39.0,4.404034
1,1.002979,0.865375,1.502164,1,49,52.0,12.915348
7,-0.996980,1.495134,1.268540,1,38,46.0,1.871428
10,1.002979,-0.106534,0.545911,1,34,45.0,-0.496970
16,-0.996980,0.043034,1.428732,1,30,39.0,2.610159


Uma pergunta imediata que surge é: isso não anula o propósito do pareamento? Se eu tiver que executar uma regressão linear de qualquer maneira, por que não uso apenas isso, em vez deste modelo complicado. Isso é um ponto válido, então vamos dedicar um tempo para respondê-lo.

![img](./data/img/matching/ubiquitous-ols.png)

Em primeiro lugar, esta regressão linear que estamos ajustando não extrapola na dimensão do tratamento para obter o efeito do tratamento. Em vez disso, seu objetivo é apenas corrigir o viés. A regressão linear aqui é local, no sentido de que não tenta ver como seria o tratado se parecesse com o não tratado. Não faz nenhuma dessa extrapolação. Isso fica por conta da parte de pareamento. A essência do estimador ainda é a componente de pareamento. O ponto que quero destacar aqui é que OLS é secundário para este estimador.

O segundo ponto é que o pareamento é um estimador não paramétrico. Ele não assume linearidade ou qualquer tipo de modelo paramétrico. Como tal, é mais flexível do que a regressão linear e pode funcionar em situações em que a regressão linear não funcionará, ou seja, aquelas em que a não linearidade é muito forte.

Isso significa que você deveria usar apenas o pareamento? Bem, essa é uma pergunta difícil. Alberto Abadie argumenta que sim, você deveria. É mais flexível e, uma vez que você tem o código, igualmente simples de executar. Eu não estou totalmente convencido disso. Por exemplo, Abadie passou muito tempo estudando e desenvolvendo o estimador (sim, ele é um dos cientistas que contribui para o pareamento ser o que é), então ele obviamente está pessoalmente comprometido com o método. Em segundo lugar, há algo na simplicidade da regressão linear que você não vê no pareamento. A matemática da derivada parcial de "mantendo tudo o mais constante" é muito mais fácil de entender com a regressão linear do que com o pareamento. Mas isso é apenas minha preferência. Para ser honesto, não há uma resposta clara para esta pergunta. De qualquer forma, de volta ao nosso exemplo.

Com a fórmula de correção de viés, obtenho a seguinte estimativa de ATE.

In [16]:
np.mean((2*predicted["medication"] - 1)*((predicted["recovery"] - predicted["match"])-predicted["bias_correct"]))

-7.36266090614141

É claro que também precisamos criar um intervalo de confiança em torno dessa medição, mas já chega de teoria matemática agora. Na prática, podemos simplesmente usar o código de outra pessoa e importar um estimador de pareamento. Aqui está um exemplo da biblioteca [causalinference](https://github.com/laurencium/causalinference).

In [17]:
from causalinference import CausalModel

cm = CausalModel(
    Y=med["recovery"].values, 
    D=med["medication"].values, 
    X=med[["severity", "age", "sex"]].values
)

cm.est_via_matching(matches=1, bias_adj=True)

print(cm.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -7.709      0.609    -12.649      0.000     -8.903     -6.514
           ATC     -6.665      0.246    -27.047      0.000     -7.148     -6.182
           ATT     -9.679      1.693     -5.717      0.000    -12.997     -6.361



Finalmente, podemos afirmar com confiança que nosso medicamento realmente reduz o tempo que alguém passa no hospital. A estimativa de ATE é apenas um pouco menor do que a minha, devido à diferença na quebra de empates nas correspondências entre a implementação de knn do sklearn e o pacote causalinference em Python.

Antes de encerrar este tópico, eu queria abordar um pouco mais a causa do viés no pareamento. Vimos que o pareamento é enviesado quando a unidade e sua correspondência não são tão semelhantes. Mas o que faz com que elas sejam tão diferentes?


## A Maldição da Dimensionalidade

Como acontece, a resposta é bastante simples e intuitiva. É fácil encontrar pessoas que coincidem em algumas características, como sexo. Mas se adicionarmos mais características, como idade, renda, cidade de nascimento e assim por diante, torna-se cada vez mais difícil encontrar correspondências. Em termos mais gerais, quanto mais características tivermos, maior será a distância entre unidades e suas correspondências.

Isso não prejudica apenas o estimador de pareamento. Isso está relacionado ao estimador de subclassificação que vimos anteriormente. No exemplo fictício do medicamento com homem e mulher, foi bastante fácil construir o estimador de subclassificação. Isso ocorreu porque tínhamos apenas 2 células: homem e mulher. Mas o que aconteceria se tivéssemos mais? Digamos que temos 2 características contínuas como idade e renda e conseguimos discretizá-las em 5 intervalos cada. Isso nos daria 25 células, ou $5^2$. E se tivéssemos 10 covariáveis com 3 intervalos cada? Não parece muito, certo? Bem, isso nos daria 59049 células, ou $3^{10}$. É fácil ver como isso pode sair rapidamente de proporção. Este é um fenômeno presente em toda a ciência de dados, chamado de **A Maldição da Dimensionalidade**!!!

![img](./data/img/curse-of-dimensionality.jpg)
Image Source: https://deepai.org/machine-learning-glossary-and-terms/curse-of-dimensionality

Apesar de seu nome assustador e pretensioso, isso significa apenas que o número de pontos de dados necessários para preencher um espaço de características cresce exponencialmente com o número de características ou dimensões. Portanto, se forem necessários X pontos de dados para preencher o espaço de, digamos, 3 características, são necessários pontos exponencialmente mais para preencher o espaço de 4 características.

No contexto do estimador de subclassificação, a maldição da dimensionalidade significa que ele sofrerá se tivermos muitas características. Muitas características implicam em múltiplas células em X. Se houver múltiplas células, algumas delas terão muito poucos dados. Algumas delas podem até ter apenas tratados ou apenas controles, então não será possível estimar o ATE ali, o que quebraria nosso estimador. No contexto do pareamento, isso significa que o espaço de características será muito esparsamente preenchido e as unidades estarão muito distantes umas das outras. Isso aumentará a distância entre as correspondências e causará problemas de viés.

Quanto à regressão linear, na verdade ela lida bastante bem com esse problema. O que ela faz é projetar todas as características X em uma única, a dimensão Y. Em seguida, ela faz comparações entre tratamento e controle nessa projeção. Portanto, de alguma forma, a regressão linear realiza algum tipo de redução de dimensionalidade para estimar o ATE. É bastante elegante.

A maioria dos modelos causais também possui alguma maneira de lidar com a maldição da dimensionalidade. Eu não vou ficar me repetindo, mas é algo que você deve ter em mente ao analisá-los. Por exemplo, ao lidarmos com escores de propensão na próxima seção, tente ver como isso resolve esse problema.

## Conceitos-chave

Começamos esta seção entendendo o que a regressão linear faz e como ela pode nos ajudar a identificar relações causais. Em particular, compreendemos que a regressão pode ser vista como a divisão do conjunto de dados em células, calculando o ATE em cada célula e, em seguida, combinando o ATE da célula em um único ATE para todo o conjunto de dados.

A partir daí, derivamos um estimador de inferência causal muito geral com subclassificação. Vimos como esse estimador não é muito útil na prática, mas nos deu algumas ideias interessantes sobre como abordar o problema da estimação da inferência causal. Isso nos deu a oportunidade de falar sobre o estimador de pareamento.

O pareamento controla os fatores de confusão ao analisar cada unidade tratada e encontrar uma unidade não tratada que seja muito semelhante a ela, e vice-versa para as unidades não tratadas. Vimos como implementar esse método usando o algoritmo KNN e também como corrigi-lo usando regressão. Por fim, discutimos a diferença entre pareamento e regressão linear. Vimos como o pareamento é um estimador não paramétrico que não depende da linearidade da maneira como a regressão linear faz.

Por fim, mergulhamos no problema de conjuntos de dados de alta dimensão e vimos como os métodos de inferência causal podem sofrer com isso.

## Referências

Gosto de pensar nesta série inteira como uma homenagem a Joshua Angrist, Alberto Abadie e Christopher Walters por sua incrível aula de Econometria. A maioria das ideias aqui foram tiradas de suas aulas na *American Economic Association*. Assisti-las é o que está me mantendo são durante este difícil ano de 2020.
* [Cross-Section Econometrics](https://www.aeaweb.org/conference/cont-ed/2017-webcasts)
* [Mastering Mostly Harmless Econometrics](https://www.aeaweb.org/conference/cont-ed/2020-webcasts)

Também gostaria de referenciar os livros incríveis de Angrist. Eles me mostraram que Econometria, ou 'Métricas, como eles chamam, não é apenas extremamente útil, mas também profundamente divertida.

* [Mostly Harmless Econometrics](https://www.mostlyharmlesseconometrics.com/)
* [Mastering 'Metrics](https://www.masteringmetrics.com/)

Finalmente, gostaria de referenciar o livro de Miguel Hernan e Jamie Robins. Tem sido meu fiel companheiro nas questões mais espinhosas de inferência causal que tive que responder.

* [Causal Inference Book](https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/)

![img](./data/img/poetry.png)


## Contribua

"Inferência Causal para os Corajosos e Verdadeiros" é um material de código aberto sobre inferência causal, a estatística da ciência. Seu objetivo é ser acessível monetariamente e intelectualmente. Ele utiliza apenas software gratuito baseado em Python.
Se você encontrou valor neste livro e deseja apoiá-lo, por favor, vá para o [Patreon](https://www.patreon.com/causal_inference_for_the_brave_and_true). Se você não estiver pronto para contribuir financeiramente, também pode ajudar corrigindo erros, sugerindo edições ou dando feedback sobre trechos que não compreendeu. Acesse o repositório do livro e abra uma issue na [versão em inglês](https://github.com/matheusfacure/python-causality-handbook/issues) ou na [versão em português](https://github.com/rdemarqui/python-causality-handbook-ptbr/issues). Por fim, se você gostou deste conteúdo, compartilhe com outras pessoas que possam achar útil e dê uma estrela no GitHub na [versão em inglês](https://github.com/matheusfacure/python-causality-handbook/stargazers) e na [versão em português](https://github.com/rdemarqui/python-causality-handbook-ptbr/stargazers).

---

<div align="center">
<a href="09-Non-Compliance-and-LATE.ipynb"><-- Anterior</a>  
<a href="00-Summary.ipynb">| Sumário |</a>  
<a href="11-Propensity-Score.ipynb">Próximo --></a>  
</div>